In [1]:
wandb_features = """Weights & Biases (W&B) is a machine learning experiment tracking tool that helps you track, visualize, and analyze your experiments. It provides a number of features that make it a valuable tool for ML practitioners, including:

Lightweight experiment tracking: W&B is a lightweight tool that can be easily integrated into your existing ML workflow. It does not require any changes to your code, and it can be used with a variety of ML frameworks.
Collaborative dashboards: W&B allows you to collaborate with others on your experiments. You can share your dashboards and reports with your team, and you can even collaborate on the same experiment in real time.
Artifacts: W&B allows you to store and share artifacts from your experiments, such as code, data, and models. This makes it easy to reproduce your experiments and to share your results with others.
Interactive data visualization: W&B provides a variety of interactive data visualization tools that allow you to explore your experiment data in depth. You can visualize your metrics, hyperparameters, and code changes over time, and you can create custom dashboards to track your progress.
Hyperparameter optimization: W&B can help you optimize your hyperparameters using a variety of techniques, such as grid search and Bayesian optimization. This can help you find the best hyperparameters for your model and improve your model performance.
Overall, W&B is a powerful tool that can help you track, visualize, and analyze your ML experiments. It is a valuable tool for ML practitioners who want to improve their experimental workflow and get the most out of their data.

Here are some additional features of W&B Experiment Tracking:

Sweeps: Sweeps allow you to automate the hyperparameter optimization process. You can define a range of values for each hyperparameter, and W&B will run your experiment for each combination of hyperparameters. This can save you a lot of time and effort, and it can help you find the best hyperparameters for your model more quickly.
Models: W&B allows you to store and share your models. You can upload your models to W&B, and you can share them with others. This makes it easy to deploy your models to production and to share your models with others.
Reports: W&B allows you to create reports of your experiments. These reports can be used to document your experiments and to share your results with others.
W&B Experiment Tracking is a powerful tool that can help you track, visualize, and analyze your ML experiments. It is a valuable tool for ML practitioners who want to improve their experimental workflow and get the most out of their data."""

tensorboard_features = """TensorBoard Experiment Tracking is a tool that helps you track, visualize, and analyze your machine learning experiments. It provides a number of features that make it a valuable tool for ML practitioners, including:

Tracking and visualizing metrics: TensorBoard allows you to track and visualize a variety of metrics from your experiments, such as loss, accuracy, and AUC. You can also visualize the distribution of your data and the relationships between different variables.
Visualizing the model graph: TensorBoard allows you to visualize the model graph, which shows the relationships between different layers in your model. This can help you understand how your model works and identify potential problems.
Projecting embeddings: TensorBoard allows you to project embeddings to a lower dimensional space. This can help you visualize the relationships between different data points and identify clusters of similar data.
Profiling TensorFlow programs: TensorBoard allows you to profile TensorFlow programs, which can help you identify bottlenecks in your code.
Sharing experiments: TensorBoard allows you to share your experiments with others. This can be helpful for collaborating with others on experiments or for documenting your experiments.
Overall, TensorBoard Experiment Tracking is a powerful tool that can help you track, visualize, and analyze your machine learning experiments. It is a valuable tool for ML practitioners who want to improve their experimental workflow and get the most out of their data.

Here are some additional features of TensorBoard Experiment Tracking:

Collections: Collections allow you to organize your experiment data into groups. This can help you keep track of your data and make it easier to find the data you need.
Experiment tags: Experiment tags allow you to add metadata to your experiments. This can help you keep track of different versions of your experiments and identify important experiments.
Remote logging: TensorBoard Experiment Tracking can be used to log data to a remote server. This can be helpful if you want to share your experiment data with others or if you want to store your experiment data in the cloud.
TensorBoard Experiment Tracking is a powerful tool that can help you track, visualize, and analyze your machine learning experiments. It is a valuable tool for ML practitioners who want to improve their experimental workflow and get the most out of their data."""

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA

llm = OpenAI(
    model="text-davinci-003",
    temperature=0
)

In [5]:
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size = 1000,
    chunk_overlap  = 0,
)

embeddings = OpenAIEmbeddings()

documents = text_splitter.create_documents([wandb_features, tensorboard_features], metadatas=[{"source": "instagram"}, {"source": "snapchat"}])

document_db = Chroma.from_documents(documents, embeddings)

Created a chunk of size 1400, which is longer than the specified 1000
Created a chunk of size 1303, which is longer than the specified 1000


In [6]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)

In [7]:
vectorstore_info = VectorStoreInfo(
    name="document_db",
    description="A vectorstore containing the wandb and tensorboard features database.",
    vectorstore=document_db,
)
toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info)
agent_executor = create_vectorstore_agent(llm=llm, toolkit=toolkit, verbose=True)

In [16]:
res = agent_executor.run("""\
compare the features of wandb and tensorboard. what are the differences? return the output in the following format: 
+-----------+--------------------+-------------+
| Sr. No.   | Weights and Biases | TensorBoard |
+-----------+--------------------+-------------+
| feature 1 |                    |             |
+-----------+--------------------+-------------+
| feature 2 |                    |             |
+-----------+--------------------+-------------+ """)



> Entering new AgentExecutor chain...
 I need to compare the features of wandb and tensorboard
Action: document_db
Action Input: compare the features of wandb and tensorboard
Observation:  TensorBoard Experiment Tracking provides features such as tracking and visualizing metrics, visualizing the model graph, projecting embeddings, profiling TensorFlow programs, and sharing experiments. W&B provides features such as lightweight experiment tracking, collaborative dashboards, artifacts, interactive data visualization, and hyperparameter optimization.
Thought: I now know the final answer
Final Answer: +-----------+--------------------+-------------+
| Sr. No.   | Weights and Biases | TensorBoard |
+-----------+--------------------+-------------+
| Experiment Tracking | Lightweight experiment tracking | Tracking and visualizing metrics, visualizing the model graph, projecting embeddings, profiling TensorFlow programs, and sharing experiments |
+-----------+--------------------+-----------

In [17]:
print(res)

+-----------+--------------------+-------------+
| Sr. No.   | Weights and Biases | TensorBoard |
+-----------+--------------------+-------------+
| Experiment Tracking | Lightweight experiment tracking | Tracking and visualizing metrics, visualizing the model graph, projecting embeddings, profiling TensorFlow programs, and sharing experiments |
+-----------+--------------------+-------------+
| Dashboards | Collaborative dashboards | - |
+-----------+--------------------+-------------+
| Artifacts | Artifacts | - |
+-----------+--------------------+-------------+
| Data Visualization | Interactive data visualization | - |
+-----------+--------------------+-------------+
| Hyperparameter Optimization | Hyperparameter optimization | - |
+-----------+--------------------+-------------+
